# Julia for Economists

## Code for the Go-Fast

### Cameron Pfiffer (cpfiffer@stanford.edu)

## Introduction

- I'm Cameron Pfiffer, a finance PhD candidate from the University of Oregon.
- Visiting at Stanford
- This is the fourth in a series of workshops on Julia for economists
- Past lectures/recordings/notes are in the GitHub repo: https://github.com/cpfiffer/julia-bootcamp-2022

## Format

- The sessions are two hours now! 
- Today is all lecture! There's going to be a ton of examples for us to walk through together.
- Ask questions when needed -- please interrupt me!
- I assume you have Julia and some text editor evailable to you. If you don't have it, please download a Julia binary here: https://julialang.org/downloads/

## Topic map

- What the heck is a compiler?
- Performant Julia! How do you write Julia code that (reliably) goes fast?
- Tracking memory allocation
- Tackling type instability
- Profiling & code analysis tools (Traceur, Profile, ProfileViews, BenchmarkTools, etc.)

## Citations

Many of these tips come from the excellent [Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/) guide on the Julia docs.

## The Julia compiler

A _compiler_ is a thing that takes human-readable code and turns it into machine-readable code. Let's take a look at one way that the compiler does this by making a really simple function to increment a value by 1.

In [550]:
# Write test_function


In [1]:
test_function(x) = x + 1

test_function (generic function with 1 method)

## The `code_typed` macro

`code_typed` takes a function call and returns the Julia-typed function. This is a good way to analyze what Julia thinks about your function! More on its use later.

In [2]:
@code_typed test_function(1.0)

CodeInfo(
1 ─ %1 = Base.add_float(x, 1.0)::Float64
└──      return %1
) => Float64

Julia will take this and give it to [LLVM](https://en.wikipedia.org/wiki/LLVM), which is the next level down in the compiler chain. LLVM is a language-independent compiler architecture tool that is used by many languages, not just Julia.

In [3]:
@code_llvm test_function(1.0)

;  @ In[1]:1 within `test_function`
define double @julia_test_function_1300(double %0) #0 {
top:
; ┌ @ promotion.jl:379 within `+` @ float.jl:399
   %1 = fadd double %0, 1.000000e+00
; └
  ret double %1
}


Lastly, the LLVM code goes to native code, which is machine code specialized to your system architecture:

In [5]:
@code_native test_function(1.0)

	.text
; ┌ @ In[1]:1 within `test_function`
	movabsq	$.rodata.cst8, %rax
; │┌ @ promotion.jl:379 within `+` @ float.jl:399
	vaddsd	(%rax), %xmm0, %xmm0
; │└
	retq
	nop
; └


## Specialization -- different inputs, different code!

In [10]:
@code_native debuginfo=:none test_function(1.0)

	.text
	movabsq	$.rodata.cst8, %rax
	vaddsd	(%rax), %xmm0, %xmm0
	retq
	nop


In [14]:
@code_native debuginfo=:none test_function(1)

	.text
	leaq	1(%rdi), %rax
	retq
	nopw	%cs:(%rax,%rax)


This gets ridiculous if you get outside numeric types:

In [17]:
@code_typed debuginfo=:none test_function('a')

CodeInfo(
1 ──        goto #25 if not true
2 ── %2   = Base.bitcast(Base.UInt32, x)::UInt32
│    %3   = Base.lshr_int(%2, 0x0000000000000018)::UInt32
│    %4   = Base.shl_int(%2, 0xffffffffffffffe8)::UInt32
│    %5   = Base.ifelse(true, %3, %4)::UInt32
│    %6   = Base.trunc_int(Int8, %5)::Int8
│    %7   = Core.sext_int(Core.Int32, %6)::Int32
│    %8   = Core.sext_int(Core.Int64, %7)::Int64
│    %9   = Base.sle_int(0, %8)::Bool
└───        goto #25 if not %9
3 ── %11  = Base.sext_int(Int64, %7)::Int64
│    %12  = Base.add_int(%11, 1)::Int64
│    %13  = Base.sle_int(0, %12)::Bool
└───        goto #5 if not %13
4 ── %15  = Base.slt_int(%12, 0)::Bool
│    %16  = Base.bitcast(UInt64, %12)::UInt64
│    %17  = Base.ult_int(%16, 0x0000000000000080)::Bool
│    %18  = Base.or_int(%15, %17)::Bool
└───        goto #6
5 ──        nothing::Nothing
6 ┄─ %21  = φ (#4 => %18, #5 => false)::Bool
└───        goto #8 if not %21
7 ── %23  = Base.trunc_int(UInt32, %12)::UInt32
│    %24  = Base.shl_int(%23,

## Why am I telling you this?

Every time you give a function a new type of input, Julia will compile a new specialization! 

If the new type is concrete and precise, Julia can make a precise, fast, specialized function.

If the new type is abstract and general, Julia can't do that specialization. It can't make ultra-fast native code for you!

## Type stability

This problem is known as __type stability__. When you call the same function a lot of times with different inputs, Julia has to compile a bunch of specialized code to handle it!

This is slow, not only because compiling can be slow, but also because type-unstable functions do not benefit from the amazing compiler optimizations that you can do when you know the type of everything.

## Type stability: an example


In [77]:
bad_function(x) = x >= 5 ? 'a' : 5*2
@code_warntype bad_function(3)

MethodInstance for bad_function(::Int64)
  from bad_function(x) in Main at In[77]:1
Arguments
  #self#::Core.Const(bad_function)
  x::Int64
Body::Union{Char, Int64}
1 ─ %1 = (x >= 5)::Bool
└──      goto #3 if not %1
2 ─      return 'a'
3 ─ %4 = (5 * 2)::Core.Const(10)
└──      return %4



Type instability can end up being a huge pain if it appears __anywhere__ in another function and affects the return type of an outer function:

In [80]:
function nested_function(y)
    x = bad_function(y)
    return x + 1
end

@code_warntype nested_function(1.0)

MethodInstance for nested_function(::Float64)
  from nested_function(y) in Main at In[80]:1
Arguments
  #self#::Core.Const(nested_function)
  y::Float64
Locals
  x::Union{Char, Int64}
Body::Union{Char, Int64}
1 ─      (x = Main.bad_function(y))
│   %2 = (x + 1)::Union{Char, Int64}
└──      return %2



## Type instability

Type stability is loosely defined as when the type of a function's output is only defined by the __type__ of the input, and not the __value__ of the input.

When Julia doesn't know the concrete type of a thing, the code touching that thing gets worse.

In [103]:
maybe_zero(x) = x >= 0 ? 0 : x
@code_warntype maybe_zero(1.0)

MethodInstance for maybe_zero(::Float64)
  from maybe_zero(x) in Main at In[103]:1
Arguments
  #self#::Core.Const(maybe_zero)
  x::Float64
Body::Union{Float64, Int64}
1 ─ %1 = (x >= 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x



When we call this a bunch of times, the type instability _really_ starts to kick in:

In [111]:
xs = randn(1_000)
@benchmark map(maybe_zero, xs)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  10.548 μs …  2.887 ms  ┊ GC (min … max): 0.00% … 98.98%
 Time  (median):     15.342 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.906 μs ± 56.549 μs  ┊ GC (mean ± σ):  6.96% ±  2.21%

     ▄      ██▆▄▄                                              
  ▂▆▅██▄▃▃▂▃█████▆▃▃▂▂▃▃▄▅▆▄▄▃▃▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁ ▃
  10.5 μs         Histogram: frequency by time        32.6 μs <

 Memory estimate: 23.64 KiB, allocs estimate: 499.

Let's fix it by making sure that the type of the output is the same regardless of whether $x \ge 0$. A handy function is `zero(thing)`, which will attempt to return a value of zero in the same type as `thing`.

In [112]:
better_zero(x) = x >= 0 ? zero(x) : x
@code_warntype better_zero(1.0)

MethodInstance for better_zero(::Float64)
  from better_zero(x) in Main at In[112]:1
Arguments
  #self#::Core.Const(better_zero)
  x::Float64
Body::Float64
1 ─ %1 = (x >= 0)::Bool
└──      goto #3 if not %1
2 ─ %3 = Main.zero(x)::Core.Const(0.0)
└──      return %3
3 ─      return x



Now, see how much faster this is! Fewer allocations, more consistent timing, and lower median/mean times.

## How to handle type instability

- Use `@code_typed` to chase down type instabilities.
- Use `zero(returntype)` to get correct zero for your type


## Traceur.jl

A _really_ good tool to use is `Traceur`, a package which flags a bunch of common performance issues in Julia on your behalf. 

To use it, just call `@trace` right before your function call and it'll give you a detailed report.

Install it with `] add Traceur` or `import Pkg; Pkg.add("Traceur")`.

In [484]:
using Traceur

# Define a type unstable function
me_badman(x) = x <= 5 ? 5 : x
@trace me_badman(5.0)

┌ Warning: me_badman returns Union{Float64, Int64}
└ @ In[484]:4


5

## Concrete typing of structs and arrays

There are a bunch of super types that are intended to describe lots of types of variables. For example, the `String` `"howdy partner"` and the `Int` `12` are both of subtype `Any`.

When we put them together in an array, the array has to have an element type that all elements are subtypes of.

In [31]:
# Create an array with lots of different types
# Note that the element type here is Any!
arr = [12, "howdy partner", 3.9]
eltype(arr)

Any

## Abstract typed arrays are bad

Arrays that are not specifically typed can be _really inefficient_, because Julia has to alocate memory assuming you can put literally anything in them. 

If you tell it an array will _only_ hold `Float64`, it can assign 64-bit storage for each element inside it, and you benefit from all kinds of fancy math specializations.

In [36]:
# Bad and slow array of `Real`s
matrix = Matrix{Real}(randn(300, 300))
@time matrix * matrix;

# Good and fast array of Float64
matrix = Matrix{Float64}(randn(300, 300))
@time matrix * matrix;

  1.370715 seconds (54.27 M allocations: 828.781 MiB, 23.88% gc time)
  0.001316 seconds (2 allocations: 703.172 KiB)


You might be thinking 

> Gosh Cameron, I'd never do this! I'm a top-notch code cowboy.

and you _might_ be right. 

However! I see this in others, and I often screw up and do this myself:

```julia
array = []

for something in an_iterator
    . . .
    push!(array, nifty_function(something))
end
```

What's wrong here?

![](https://c.tenor.com/m6GfTVTJqN8AAAAd/bad-bad-boy.gif)

The type of the array defaults to `Any`, because I haven't provided any other information! 

If you know for a fact it's going to be of a certain type, type it ahead of time:

```julia
array = Float64[]
```

or if you don't know it's exactly `Float64` but you have something available you know is of the type you will be putting in the array, you can do

```julia
array = typeof(a_thing)[]
```

or, if you know the size, just preallocate the whole array with

```julia
array = Array{typeof(a_thing)}(undef, size_of_array)
```

In [56]:
# Let's see an example of how to operationalize this:
do_stuff(x) = (x=x, sin_x=sin(x), str_x=string(x))
println(typeof(do_stuff(1.0)))

function bad_arrays(xs)
    arr = []
    for x in xs
        push!(arr, do_stuff(x))
    end
    return mapreduce(nt -> nt.sin_x, +, arr)
end

function good_arrays(xs)
    # Get the exact type of the thing you're going to store!
    first_val = do_stuff(xs[1])
    
    # Now julia can infer the type of the array
    arr = [first_val]
    for x in xs[2:end]
        push!(arr, do_stuff(x))
    end
    return mapreduce(nt -> nt.sin_x, +, arr)
end

xs = randn(100)
b1 = @benchmark bad_arrays(xs)
b2 = @benchmark good_arrays(xs)

display(b1)
display(b2)

NamedTuple{(:x, :sin_x, :str_x), Tuple{Float64, Float64, String}}


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  16.582 μs …   3.620 ms  ┊ GC (min … max):  0.00% … 99.21%
 Time  (median):     18.638 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   24.414 μs ± 113.228 μs  ┊ GC (mean ± σ):  15.33% ±  3.29%

  ▁▅███▇▇▆▃▁                           ▁▁▁▂▂▂▁▁▁▁ ▁            ▂
  ███████████▇▇▇▇▇█▇▆▇▇▇▇▇▇▇▆▆▆▇▆▇▆▇▇███████████████▇███▇█▇▆▅▇ █
  16.6 μs       Histogram: log(frequency) by time      41.8 μs <

 Memory estimate: 50.33 KiB, allocs estimate: 503.

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  13.640 μs …  2.681 ms  ┊ GC (min … max):  0.00% … 98.97%
 Time  (median):     15.657 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   20.150 μs ± 76.889 μs  ┊ GC (mean ± σ):  11.94% ±  3.12%

  ▁▄▆▇█▇▆▃▁▁                           ▁ ▁▁▁▁▁                ▂
  ██████████▇▇▆▇▇▇█████▇███▇▇▇▆▆▆▇▆▇█████████████▇██▇█▇▇▇▆▆▆▆ █
  13.6 μs      Histogram: log(frequency) by time      38.2 μs <

 Memory estimate: 48.53 KiB, allocs estimate: 206.

In [57]:
function gooder_arrays(xs)
    # Get the exact type of the thing you're going to store!
    first_val = do_stuff(xs[1])
    
    # Now julia can infer the type of the array
    arr = Array{typeof(first_val)}(undef, length(xs))
    arr[1] = first_val
    
    for i in 2:length(xs)
        arr[i] = do_stuff(xs[i])
    end
    
    return mapreduce(nt -> nt.sin_x, +, arr)
end

@benchmark gooder_arrays(xs)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  12.547 μs …  2.762 ms  ┊ GC (min … max):  0.00% … 99.19%
 Time  (median):     14.276 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   18.636 μs ± 78.285 μs  ┊ GC (mean ± σ):  13.19% ±  3.13%

  ▂▆▇██▇▅▂                            ▂▁▁▂▂▂▁▁▁               ▂
  █████████▆▆▆▇▆▆▆▇▆▆▆▅▆▇▆▆▆▆▆▆▆▆▅▆███████████████▇▆▇▇▇▆▆▇▆▆▅ █
  12.5 μs      Histogram: log(frequency) by time      36.3 μs <

 Memory estimate: 44.70 KiB, allocs estimate: 202.

In [60]:
function best_arrays(xs)
    # map can handle all the typing information on our behalf!
    arr = map(do_stuff, xs)
    return mapreduce(nt -> nt.sin_x, +, arr)
end

@benchmark best_arrays(xs)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  12.260 μs …  2.547 ms  ┊ GC (min … max):  0.00% … 98.81%
 Time  (median):     13.976 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   18.424 μs ± 77.355 μs  ┊ GC (mean ± σ):  13.19% ±  3.13%

  ▁▅▇██▇▄▁                           ▁▂▁▂▂▁▁                  ▂
  ████████▇▄▅▆▅▅▆▆▆▆▆▇▆▇▇▆▇▇▆▆▆▆▆▆▆███████████▇█▇█▇▇▇▇▇▇▆▇▆▆▆ █
  12.3 μs      Histogram: log(frequency) by time      36.6 μs <

 Memory estimate: 44.70 KiB, allocs estimate: 202.

## Abstract types and structs

We'll often have structs we write as programmers to contain results, set model configurations, etc. 

In [61]:
struct StructWithField
    a::Float64
    b::Int
    c::String
end

Sometimes we don't know ahead of time what the type is going to be, or we are too lazy to type them. What people do in these cases is something like

In [62]:
struct BadlyTypedStruct
    a::Real
    b::Integer # Integer is an abstract type for Int
    c::Any
    d # Leaving this blank means ::Any
end

These structs end up being slow and clunky for the same reason that non-concrete arrays are slow.

The solution? Parametric typing!

## Parametric types

Parametric types are a way of saying "I know I want this struct to be typed, but I don't know it yet".

The way we do this in Julia is to write

```julia
struct TypeName{TypeVariable}
    field::TypeVariable
end
```

When we construct `TypeName` and give it an input, it will set the type `TypeVariable` to be whatever we put in `field`, which makes `TypeName` stable & concrete (provided the input is concretely typed).

In [63]:
struct ParametricStruct{A<:Real,B<:Int,C}
    field_one::A 
    field_two::B
    field_three::C
end

ParametricStruct(1.0, 3, randn(10))

ParametricStruct{Float64, Int64, Vector{Float64}}(1.0, 3, [0.018768954890408206, -1.381636864324282, -2.135245727189524, -1.9974817157338216, -1.3252132704432988, -0.14771094570042914, -0.32371026207412956, 1.0452737401533858, -0.3531015093580762, 0.9155007396857043])

## Don't change the types of things during runtime

If you assign a variable a value, try to change that variable only to other values of the same type. I.e. don't do stuff like:

In [487]:
function type_changer()
    x = "hello" # Start as a string
    for i in 1:10
        random_draw = randn()
        if random_draw >= 0
            # Changes x to an integer!
            x = random_draw
        else
            # Add the new value as a string to the end of x
            x = string(x, random_draw)
        end
    end
    return x
end

@code_warntype type_changer()

MethodInstance for type_changer()
  from type_changer() in Main at In[487]:1
Arguments
  #self#::Core.Const(type_changer)
Locals
  @_2::Union{Nothing, Tuple{Int64, Int64}}
  x::Union{Float64, String}
  i::Int64
  random_draw::Float64
Body::Union{Float64, String}
1 ─       (x = "hello")
│   %2  = (1:10)::Core.Const(1:10)
│         (@_2 = Base.iterate(%2))
│   %4  = (@_2::Core.Const((1, 1)) === nothing)::Core.Const(false)
│   %5  = Base.not_int(%4)::Core.Const(true)
└──       goto #7 if not %5
2 ┄ %7  = @_2::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (random_draw = Main.randn())
│   %11 = (random_draw >= 0)::Bool
└──       goto #4 if not %11
3 ─       (x = random_draw)
└──       goto #5
4 ─       (x = Main.string(x, random_draw))
5 ┄       (@_2 = Base.iterate(%2, %9))
│   %17 = (@_2 === nothing)::Bool
│   %18 = Base.not_int(%17)::Bool
└──       goto #7 if not %18
6 ─       goto #2
7 ┄       return x



In [488]:
# Or, using Traceur:
@trace type_changer()

┌ Warning: dynamic dispatch to (Core.apply_type(Base.Val, _2))()
└ @ essentials.jl:701
┌ Warning: dynamic dispatch to Base.ArgumentError("negative maxsize")
└ @ iobuffer.jl:92
┌ Warning:  is assigned as Int64
└ @ ryu/shortest.jl:313
┌ Warning:  is assigned as Nothing
└ @ ryu/shortest.jl:313
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64, Int64}}
└ @ ryu/shortest.jl:408
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64, Int64}}
└ @ ryu/shortest.jl:411
┌ Warning: c is assigned as UInt32
└ @ ryu/shortest.jl:359
┌ Warning: c is assigned as UInt32
└ @ ryu/shortest.jl:374
┌ Warning: c is assigned as UInt32
└ @ ryu/shortest.jl:383
┌ Warning: c is assigned as UInt32
└ @ ryu/shortest.jl:389
┌ Warning: c is assigned as Int64
└ @ ryu/shortest.jl:472
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64, Int64}}
└ @ ryu/shortest.jl:339
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64, Int64}}
└ @ ryu/shortest.jl:341
┌ Warning: dynamic dispatch to Base.AssertionError("0 < pos <= len

0.7006771111649364

## Break up big functions into multiple smaller functions

When you add more functions, Julia is able to specialize each function better. You get lots of free speed (and remove roadblocks for yourself) when you can let the compiler be smart for you.

It's also considered good form! It's much easier to debug one small component function than a gigantic super function.

In [175]:
function fill_random(n)
    v = rand(Bool) ? Vector{Float64}(undef, n) : Vector{Int}(undef, n)
    for i in 1:n
        v[i] = rand(eltype(v))
    end
    return v
end

@benchmark fill_random(1000)

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.089 μs … 552.062 μs  ┊ GC (min … max): 0.00% … 97.78%
 Time  (median):     4.009 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.511 μs ±  12.679 μs  ┊ GC (mean ± σ):  7.31% ±  2.59%

        ▁▆▂█▂▅▂▃▄ ▄  ▃ ▂                                       
  ▁▁▂▃▅▆████████████▇█▂█▂▆▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  3.09 μs         Histogram: frequency by time        7.11 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

In [174]:
# rfill! can be specialized for Float64 or Int!
function rfill!(v::Vector{T}) where T
    for i in eachindex(v)
        v[i] = rand(T)
    end
end

function fill_random_compartment(n)
    v = rand(Bool) ? Vector{Float64}(undef, n) : Vector{Int}(undef, n)
    rfill!(v)
    return v
end

@benchmark fill_random_compartment(1000)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.672 μs … 569.070 μs  ┊ GC (min … max): 0.00% … 97.54%
 Time  (median):     3.131 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.910 μs ±  13.755 μs  ┊ GC (mean ± σ):  9.70% ±  2.76%

  ▃▄▃▇██▆▅▄▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁ ▁▁      ▁▁                      ▂
  ████████████████████████████████▇██████████▇█▇▇▆▅▅▄▅▅▅▄▅▄▅▄ █
  2.67 μs      Histogram: log(frequency) by time      7.75 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

## Column order vs. row order

When Julia stores arrays in memory, values in the same column are next to each other. In layman's terms, it's faster to go to the next row within a column rather than to go to the next column within a row.

In [192]:
function rowsum(M)
    s = zero(eltype(M))
    for row in 1:size(M,1)
        for col in 1:size(M,2)
            s += M[row, col]
        end
    end
    return s
end

function colsum(M)
    s = zero(eltype(M))
    for col in 1:size(M,2) # The only difference here is that col goes first!
        for row in 1:size(M,1)
            s += M[row, col]
        end
    end
    return s
end

M = randn(10_000, 10_000)

@time rowsum(M);
@time colsum(M);

  1.647296 seconds (19.54 k allocations: 1.094 MiB, 1.31% compilation time)
  0.179503 seconds (19.54 k allocations: 1.092 MiB, 9.81% compilation time)


## Column/row order: the takeaway

When looping through columns or rows, remember that it's easier to go down a row than it is to go across a column. 

__Put rows in your inner loop, columns on your outer loop__.

## Compiler annotations

Sometimes you can give the compiler helpful hints to remove things like bounds checking, or tell the compiler that something can be cheaply parallelized at the CPU level.

`@inbounds` in front of an expression removes bounds checking. Bounds checking is there to protect you to make sure that you aren't trying to get the 11th item of a 10-element array. 

For simple loops, Julia can figure out usually that you're within bounds and won't check for you, but sometimes it can help to annotate.

It's important sometimes -- only use `@inbounds` when you know for a _fact_ that you are accessing the correct element of an array. If you screw this up, you could get silently nonsensical results.

In [454]:
function sumprod_normal(A::AbstractArray, B::AbstractArray)
    @assert size(A) == size(B)
    r = zero(eltype(A))
    for i in eachindex(A)
        r += A[i] * B[i]
    end
    return r
end

function sumprod_inbounds(A::AbstractArray, B::AbstractArray)
    @assert size(A) == size(B)
    r = zero(eltype(A))
    for i in eachindex(A)
        @inbounds r += A[i] * B[i]
    end
    return r
end



sumprod_inbounds (generic function with 1 method)

In [457]:
xs1 = randn(10_000)
xs2 = randn(10_000)
@btime sumprod_normal(xs1, xs2)
@btime sumprod_inbounds(xs1, xs2);

  19.234 μs (1 allocation: 16 bytes)
  16.116 μs (1 allocation: 16 bytes)


98.99688965175507

## `@simd`

Single instruction, multiple data is a parallelization paradigm. Essentially, you tell your processor it can do the same thing with lots of different data points, and this _can_ speed up your code.

When you use `@simd`, you are telling the compiler that the order of your operations can be done in a more or less random order. You for loop operations cannot depend on one another! No time series SIMD!

In [469]:
function sumprod_simd(A::AbstractArray, B::AbstractArray)
    @assert size(A) == size(B)
    r = zero(eltype(A))
    @simd for i in eachindex(A)
        r += A[i] * B[i]
    end
    return r
end;


In [470]:
@btime sumprod_simd(xs1, xs2);

  15.526 μs (1 allocation: 16 bytes)


You can also combine lots of performance annotations, though __do this cautiously!__

In [472]:
function sumprod_super(A::AbstractArray, B::AbstractArray)
    @assert size(A) == size(B)
    r = zero(eltype(A))
    @simd for i in eachindex(A)
        @inbounds r += A[i] * B[i]
    end
    return r
end;

In [473]:
@btime sumprod_super(xs1, xs2);

  2.515 μs (1 allocation: 16 bytes)


## Don't do stuff to global variables

You should not be using non-constant global variables anyway! As a refresher, `x` below is a global variable, because everything can access it.

In [476]:
x = "I am a bad global variable user" # Bad person!

function do_stuff()
    # x here is global, and the compiler 
    # cannot assert the stability of its type
    # in general cases.
    return x * ", please be kind to me"
end

@btime do_stuff();

  48.117 ns (1 allocation: 80 bytes)


There's two solutions here:

1. Annotate your global variable with `const`, which says "I am not going to change this variable, I promise".
2. Pass the variable in as an argument to the function.

In [477]:
const x_constant = "I am a bad global variable user"

function do_stuff_constant()
    # Now x is constant, the compiler can handle it.
    return x_constant * ", please be kind to me"
end

@btime do_stuff_constant();

  29.286 ns (1 allocation: 80 bytes)


In [479]:
# You can (and should) also pass it as an argument.
function do_stuff_arg(x_arg)
    return x_arg * ", please be kind to me"
end

@btime do_stuff_arg("I am a bad global variable user");

  28.767 ns (1 allocation: 80 bytes)


## In-place operations

If you do a ton of little matrix computations, it's often helpful to store results in a single preallocated buffer rather than continuously reallocate.

In Julia, a common way of doing this is with what are called __in-place operations__. In-place operations modify their input rather than returning a new object in memory.

## In-place operations

For example, rather than

```julia
X = A*B
```

we can do 

```julia
mul!(X, A, B)
```

which keeps allocations down. 

Oftentimes this is FREE SPEED, though not always. Benchmark your application to see if it actually helps.

In [489]:
function eigtimer()
    M = randn(50, 50)
    @btime eigvals(M)
    @btime eigvals!(M);
    return
end

eigtimer()

  4.279 μs (13 allocations: 4.81 KiB)
  3.808 μs (12 allocations: 3.94 KiB)


## Use multiple dispatch to your advantage

A common thing people might do in other languages is this:

In [494]:
function square(thing)
    if typeof(thing) <: Real
        return thing ^ 2
    elseif typeof(thing) <: Vector && eltype(thing) <: Real
        return thing * thing'
    end
end

@btime square(1.0)
@btime square([1.0, 2.0])

  0.041 ns (0 allocations: 0 bytes)
  111.885 ns (2 allocations: 176 bytes)


2×2 Matrix{Float64}:
 1.0  2.0
 2.0  4.0

The preferred approach is to break the typing into different functions:

In [495]:
square(thing::Real) = thing^2
square(thing::AbstractVector{<:Real}) = thing * thing'

@btime square(1.0)
@btime square([1.0, 2.0])

  0.038 ns (0 allocations: 0 bytes)
  112.008 ns (2 allocations: 176 bytes)


2×2 Matrix{Float64}:
 1.0  2.0
 2.0  4.0

This is not _necessarily_ a speed thing, but it can be. The example above doesn't actually have much in the way of speed improvements, but it is more attractive to write and easier to debug.

## Be careful with closures

A closure is a function that captures a variable outside the scope of the function. 

For example, the function below returns a new function that _closes_ over `r`.

In [505]:
function abmult(r::Int)
    if r < 0
        r = -r
    end
    f = x -> x * r # We close over r here -- Julia doesn't know that r is a fixed type here!
    return f
end;
@btime abmult(3)(2);

  56.313 ns (1 allocation: 16 bytes)


In [503]:
function abmult2(r0::Int)
    r::Int = r0 # Add a type annotation to a local variable
    if r < 0
        r = -r
    end
    f = x -> x * r # Now we're closing over r, a local variable with a specific type
    return f
end
@btime abmult2(3)(2);

  4.381 ns (0 allocations: 0 bytes)


In [506]:
function abmult3(r::Int)
    if r < 0
        r = -r
    end
    f = let r = r # Or, we can use a let block to copy the outer r to the closure.
            x -> x * r
    end
    return f
end
@btime abmult3(3)(2);

  0.038 ns (0 allocations: 0 bytes)


Note that the use of the `let` block above may become a part of Julia's compiler, so you potentially won't need to do this in the future.